In [ ]:
import json
import plotly.graph_objects as go
import shapely.wkt as shapely_wkt
import shapely.geometry as shapely_geom
import datetime
import pymap3d
import pandas as pd
import numpy as np


file_path = 'pi/trace.json'



In [ ]:
def loads(file_path):
    contents = open(file_path, "r").read() 
    for item in contents.strip().split('\n'):
        try:
           yield json.loads(str(item))
        except:
            # ignore malformed rows
            pass    

WESTBEACH_LATITUDE = 138.506

def split_outings(locations):
    last_idx = 0
    for idx, (v1, v2) in enumerate(zip(locations, locations[1:])):
        d1 = datetime.datetime.fromisoformat(v1["timestamp"])
        d2 = datetime.datetime.fromisoformat(v2["timestamp"])

        if (d2 - d1).total_seconds() > 1 * 60 * 60:
            yield [v for v in locations[last_idx:idx+1] if v["latitude"] != 0 and v["longitude"] != 0 and v["longitude"] < WESTBEACH_LATITUDE]
            last_idx = idx+1


data = loads(file_path)



In [ ]:


first_tack = 0
def extract_race(df, tack_headings, first_tack=0,  max_legs=None):
    max_heading = tack_headings['unwrapped'][first_tack]
    min_heading = tack_headings['unwrapped'][first_tack]

    # bouy roundings
    leg_turns = []
    legs = []
    # init in case loop doesn't run at all
    t = first_tack
    for t in range(first_tack+1, tack_headings['tack'].max() + 1):
        heading = tack_headings['unwrapped'][t]
        if heading < max_heading - leg_threshold:
            start_idx = df.loc[df.tack == first_tack].index[0]
            end_idx = df.loc[df.tack == t-1].index[-1]
            legs.append((start_idx, end_idx))

            max_heading = heading
            first_tack = t
        elif heading > min_heading + leg_threshold:
            t -= 1
            break
        else:
            max_heading = max(max_heading, heading)
            min_heading = min(min_heading, heading)
        
        if len(legs) == max_legs:
            break

    # last leg to finish
    if max_legs is None or len(legs) < max_legs:
        start_idx = df.loc[df.tack == first_tack].index[0]
        end_idx = df.loc[df.tack == t].index[-1]
        legs.append((start_idx, end_idx))
        
    return legs






In [ ]:
def extract_marks(df, legs):
    leg_turns = pd.DataFrame([df.loc[[l1[1], l2[0]], ["x", "y"]].mean() for l1, l2 in zip(legs, legs[1:])], columns=["x", "y"])

    # take the average rounding point between laps
    top_mark = leg_turns.iloc[::2][["x", "y"]].mean()
    bottom_mark = leg_turns.iloc[1::2][["x", "y"]].mean()
    
    if len(leg_turns) >= 2:
        top_diff = np.max(np.linalg.norm(leg_turns.iloc[::2][["x", "y"]] - top_mark, axis=1))
        bottom_diff = np.max(np.linalg.norm(leg_turns.iloc[1::2][["x", "y"]] - bottom_mark, axis=1))
        max_diff = max(top_diff, bottom_diff)
    else:
        max_diff = None
    marks = pd.DataFrame(dict(x=[top_mark.x, bottom_mark.x], y=[top_mark.y, bottom_mark.y]))

    return marks, max_diff


In [ ]:
# rolling mean window (samples) used to determine tacks
window = 10
# minimum heading change to identify a tack
tack_threshold = 40
# minimum heading change to identify a leg
leg_threshold = 150

# how close marks need to be considered the same mark
mark_diff_threshold = 1

def extract_races(df):
    df['unwrapped'] = np.rad2deg(np.unwrap(np.deg2rad(df["heading"])))
    df['unwrapped_mean'] = df['unwrapped'].rolling(window=window).mean()
    df[f'delta_heading'] = df['unwrapped_mean'].shift(periods=-1 * window) - df['unwrapped_mean']

    # identify legs by heading change
    not_tacking = abs(df[f'delta_heading']) < tack_threshold
    groups = ((not_tacking != not_tacking.shift()).cumsum() / 2).astype(int) - 1
    df['tack'] = groups.where(not_tacking, other=-1)

    groups = df[df.tack >= 0].groupby('tack', as_index=False)
    tack_headings = groups['unwrapped'].mean()

    races = []
    first_tack = 0
    while first_tack < tack_headings["tack"].max():
        legs = extract_race(df, tack_headings, first_tack, max_legs=5)

        if len(legs) == 5:
            marks, max_diff = extract_marks(df, legs)
            if max_diff < mark_diff_threshold:
                race = df.loc[slice(legs[0][0], legs[-1][1])].copy()

                race["leg"] = -1
                for leg, (first, last) in enumerate(legs):
                    race.loc[slice(first, last), 'leg'] = leg

                races.append((race, marks))

                first_tack = df.loc[legs[-1][1]]["tack"]

        first_tack += 1
    return races


In [ ]:
#
# Calculate mark positions and VMG
#

def do_vmg(df, marks):
    # determine upwind & downwind heading between marks
    delta = marks.loc[0] - marks.loc[1]
    upwind_heading = np.rad2deg(np.arctan2(delta["y"], delta["y"]))

    delta = marks.loc[1] - marks.loc[0]
    downwindow_heading = np.rad2deg(np.arctan2(delta["y"], delta["y"]))

    # VMG: vmg = speed * math.cos(heading - bouy_heading)
    upwind = df[df.leg % 2 == 0]
    upwind = upwind['speed'] * np.cos(np.deg2rad(upwind['heading'] - upwind_heading))

    downwind = df[df.leg % 2 == 1]
    downwind = downwind['speed'] * np.cos(np.deg2rad(downwind['heading'] - downwindow_heading))

    df["vmg"] = pd.concat([upwind, downwind]).sort_index()

    #
    # Normalize VMG - around the median, preserving outliers
    #
    upwind = df[(df.leg % 2 == 0) & (df.tack >= 0)]["vmg"]
    upwind = (upwind - upwind.median()) / upwind.mad()

    downwind = df[(df.leg % 2 == 1) & (df.tack >= 0)]["vmg"]
    downwind = (downwind - downwind.median()) / downwind.mad()

    df["vmg_norm"] = pd.concat([upwind, downwind]).sort_index()



In [ ]:
colorscale = "Turbo"
div_colorscale = "RdYlBu_r"

def plot_race(df, marks):
    fig = go.Figure()

    fig.add_trace(
        go.Scatter(
            x=marks["x"],
            y=marks["y"],
            marker = dict(size=20),
            mode="markers",
            name="marks"
        )
    )

    fig.add_trace(
        go.Scatter(
            x=df["x"],
            y=df["y"],
            marker = dict(
                color=df["speed"], 
                colorscale=colorscale,
                colorbar=dict(title='speed', len=0.5), 
            ),
            mode="markers",
            text=df["speed"],
            name="speed"
        )
    )

    fig.add_trace(
        go.Scatter(
            x=df["x"],
            y=df["y"],
            marker = dict(
                color=df["vmg"], 
                colorscale=colorscale,
                colorbar=dict(title='VMG', len=0.5), 
            ),
            mode="markers",
            text=df["vmg"],
            name="vmg"
        )
    )


    tmp = df[(df.leg % 2 == 0)]
    fig.add_trace(
        go.Scatter(
            x=tmp["x"],
            y=tmp["y"],
            marker = dict(
                color=tmp["vmg_norm"], 
                colorscale=div_colorscale, 
                cmin=-max(abs(tmp["vmg_norm"])),
                cmax=max(abs(tmp["vmg_norm"])),
                colorbar=dict(title='Up wind', len=0.5), 
            ),
            mode="markers",
            text=tmp["vmg_norm"],
            name="normalized vmg upwind"
        )
    )

    tmp = df[(df.leg % 2 == 1)]
    fig.add_trace(
        go.Scatter(
            x=tmp["x"],
            y=tmp["y"],
            marker = dict(
                color=tmp["vmg_norm"], 
                colorscale=div_colorscale, 
                cmin=-max(abs(tmp["vmg_norm"])),
                cmax=max(abs(tmp["vmg_norm"])),
                colorbar=dict(title='Down wind', len=0.5),
            ),
            mode="markers",
            text=tmp["vmg_norm"],
            name="normalized vmg downwind"
        )
    )

    fig.update_layout(
        autosize=False,
        width=1200,
        height=1200,)

    fig.show()

    return fig




In [ ]:
def races(data):
    gps_locations = [m for m in data if m["message"] == "gps"]

    for o in split_outings(gps_locations):
        df = pd.DataFrame.from_records(o, columns =['timestamp', 'latitude', 'longitude', 'heading', 'speed'])

        df['ts'] = pd.to_datetime(df['timestamp'])

        df[['latitude', 'longitude']] = np.deg2rad(df[['latitude', 'longitude']])
        mean_lat, mean_lon = df[["latitude", "longitude"]].mean()
        df['x'], df['y'], _ = pymap3d.geodetic2enu(df['latitude'], df['longitude'], 0, mean_lat, mean_lon, 0)

        # seconds since start
        df['seconds'] = (df['ts'] - df['ts'].min()).dt.total_seconds()
        df.set_index('seconds', inplace=True)

        for race in extract_races(df):
            yield race



In [ ]:
for i, r in enumerate(races(data)):
    do_vmg(*r)
    fig = plot_race(*r)

    html = fig.to_html(full_html=True, include_plotlyjs='cdn')
    open(f"race_{i}.html", "w").write(html)
